# 导入包和设置全局路径

In [1]:
import os, time, datetime
import zipfile
import lxml.etree as etree
import shutil
import numpy as np
import pandas as pd
import re, pickle, gzip
import pdb

In [2]:
import sklearn.preprocessing

In [3]:
from sklearn.decomposition import PCA

In [4]:
from sklearn.cluster import KMeans

In [5]:
import tensorflow as tf

In [6]:
import seaborn as sns

In [7]:
import matplotlib.pyplot as plt

###### 设置路径

In [8]:
strJobsDir = "../jobs_nobranch"
strZipsDir = "../zips"

# 工具函数

## 通过输入的任务文件夹返回任务文件夹中的所有解调器文件夹路径

In [9]:
def fn_getDemodDirsOfAJob(strJobDir):
    liststrDemodDirs = [os.path.join(strJobDir, strName) for strName in os.listdir(strJobDir) if "Demod" in strName]
    return liststrDemodDirs

## 解调器属性选取（更新）

检查所有的解调器状态文件发现，以下属性不是所有的解调器都存在：

DEMOD_PHASEROTATION，GLOBAL_DEMOD1STATUS，GLOBAL_TEMPSTATUS，DRU_DISKSPACE1，DRU_USEDSPACE1，DRU_FREESPACE1，DRU_USEDPERCENT1

对它们进行忽略。

目前发现的在数据集中的值只有一种的属性也全部忽略。以后可能会发现它们有多种值。

解调器的状态参数按照参数名的前缀来划分大致有3类——DPU（data packet unit），DEMOD、IFU(中频控制单元)，DRU。



# 读入任务配置文件

## 读入任务计划文件

首先清空jobs文件夹。\
WorkSch文件中存放了提前计算出的预计接收开始和结束时间。需利用这个接收开始和结束时间截取出有效时间段。\
可能有多个WorkSch文件，根据它的createdTime选择最新的WorkSch文件。

只有清华解调器和融为解调器具有反映输出速率的参数，要先把这两个解调器的zip文件路径存储到链表中去

## 读入设备列表文件
任务所用的所有设备都在列表文件里。要根据文件中对设备状态文件的属于的设备的描述提取出用于数据接收传输的设备状态文件。

# 解调器预处理描述

1. 根据任务的设备列表文件找到解调器的设备ID
2. 利用设备ID找到解调器的设备状态文件和控制文件，并保存
3. 从任务的任务计划文件中读出接收的开始和结束时间，利用接收时间段截取出有效时间段内的记录
4. 将解调器的状态参数按照信号处理流程划分为数个最小模块
5. 针对每个最小模块产生训练样本和测试样本

# 载入设备状态文件和控制文件

1. 清空原有的设备文件夹。
2. 在设备列表文件中找到解调器设备ID
3. 将相应设备状态文件载入设备文件夹

执行完成后，任务文件夹里多出如下文件：
* 任务名/
    * 解调器名/
        * raw/
            * status.csv
        * control.csv

In [62]:
# 从zip文件中解压缩具有设备的状态文件，存入raw
def fn_extractFilesFromAZip(strJobDir, strID, strZipFile):
    strZipDemodStatusFile = "Status_" + strID
    with zipfile.ZipFile(strZipFile) as oZipFile:
        for strFile in oZipFile.namelist():
            # 迭代找到zip文件里的解调器的状态文件和控制参数文件
            if "Status_" + strID in strFile:
                # 状态文件
                with oZipFile.open(strFile) as f:
                    with open(os.path.join(strJobDir, strID + "/raw/status.csv"), "wb") as f1:
                        f1.write(f.read())
            elif "Control_" + strID in strFile:
                # 控制文件
                with oZipFile.open(strFile) as f:
                    with open(os.path.join(strJobDir, strID + "/control.csv"), "wb") as f1:
                        f1.write(f.read())

for strJob in os.listdir(strJobsDir):
    strJobDir = os.path.join(strJobsDir, strJob)
    # 删除原有的解调器文件夹
    for strName in os.listdir(strJobDir):
        if "Demod" in strName:
            strDemodDir = os.path.join(strJobDir, strName)
            shutil.rmtree(strDemodDir)
    
    strZipFile = os.path.join(strZipsDir, strJob + ".zip")
    strDeviceListFile = os.path.join(strJobDir, "device_list.xml")
    oElementTree = etree.parse(strDeviceListFile)
    listDevElement = oElementTree.findall("./content/deviceList/Device")
    for oElement in listDevElement:
        strID = oElement.find("DevID").text
        if "Demod" in strID:
            # 新建解调器文件raw文件夹
            os.makedirs(os.path.join(strJobDir, strID + "/raw"))
            fn_extractFilesFromAZip(strJobDir, strID, strZipFile)

# 截取出有效接收时间范围内的记录

1. 首先清空每个解调器文件夹下的valid文件夹
2. 然后从work_sch.xml文件中读入数据接收的有效时间段
3. 根据有效时间段截取每个解调器的设备状态文件的有效记录，存入valid文件夹
执行完成后，任务文件里多出了如下文件：
* 任务名/
    * 解调器名/
        * valid/
            * status.csv

从指定任务文件夹中读取任务所使用的解调器名

# 划分描述
解调译码过程按照信号处理流程包含如下阶段：
* 中频输入
* 载波同步
* 比特同步
* 维特比译码（I/Q路）
* 帧同步（I/Q路）
* 译码和解扰（I/Q路）

新建parts文件夹，存放各部分的数据集

# 帧同步锁前

## 工具函数

### 得到解调器的正常记录index
当一条记录的两个帧同步锁都锁上的时候说明该记录在帧同步锁之前的各个阶段均正常，即帧同步锁之前的各个部分用作正常训练样本。曾经考虑对于分路帧同步的情况，如果分路中某一个同步锁锁上了，则可以认为该记录在维特比译码之前各个阶段均正常，但是在分路的情况下，只有一个分路的帧同步锁上比方说I路，不能说明在分路之前的所有阶段均正确，所以舍弃这种做法。

### 按照 index拼接样本csv
为了解决前后两部分的“生成样本”时，相同的参数在两部分的处理工作重复的问题，设计该函数，把两个dataframe按照前一个打他frame的index进行拼接

## 中频输入

### 载入数据
中频输入特征参数：DEMOD_IFLEVEL（输入电平），DEMOD_EBNOVALUE（信噪比）。\
本来还想选用DEMOD_EBNOVALUEQCHL，但是这个参数在整个数据集中取值均为零，故忽略。



添加中频输入部分参数名

### 生成样本
利用帧同步锁生成正常训练样本和异常样本。该步进行完成后，文件夹具有如下结构：
* parts/
    * sync/
        * input/
            * status.csv
            * samples/
                * normal/
                    * train/
                        * samples.csv
                * abnormal/
                    * samples.csv

### 正则化
用正则化器适应训练集，再用正则化器去正则化训练集和测试集中的数值属性。将转化的结果存入preprocessed文件夹。该步执行完后，具有的文件夹结构如下：
* input/
    * samples/
        * normal/
            * train/
                * samples.csv
                * preprocessed/
                    * samples.csv
            * test/
                * samples.csv
                * preprocessed/
                    * samples.csv
        * abnormal/
            * samples.csv
            * preprocessed/
                * samples.csv

MinMaxScaler(copy=True, feature_range=(0, 1))

## 载波同步
载波同步特征参数：DEMOD_CARRIEROFFSET（载波偏移）、DEMOD_CARRIERLOCK（载波锁）

### 载入数据
载入载波同步部分到carrier文件夹

In [176]:
listcarrierFeatures = []
listFeatures = listcarrierFeatures.copy()

In [124]:
# 建立空的文件夹carrier，将数据导入
for strJob in os.listdir(strJobsDir):
    liststrDemodDirs = fn_getDemodDirsOfAJob(os.path.join(strJobsDir, strJob))
    for strDemodDir in liststrDemodDirs:
        strSectionDir = os.path.join(strDemodDir, "parts/sync/carrier")
        if os.path.exists(strSectionDir):
            shutil.rmtree(strSectionDir)
        os.mkdir(strSectionDir)
    
        # 读入有效解调器设备状态参数
        strValidStatusFile = os.path.join(strDemodDir, "valid/status.csv")
        pdDfValidStatus = pd.read_csv(strValidStatusFile, index_col="RECTIME")
        pdDfSectionStatus = pdDfValidStatus[listFeatures]
        strSectionStatusFile = os.path.join(strSectionDir, "status.csv")
        pdDfSectionStatus.to_csv(strSectionStatusFile, index_label="RECTIME")

### 更改二元属性值


In [125]:
for strJob in os.listdir(strJobsDir):
    listDemodDirs = fn_getDemodDirsOfAJob(os.path.join(strJobsDir, strJob))
    for strDemodDir in listDemodDirs:
        pdDfSectionStatus = pd.read_csv(os.path.join(strDemodDir, "parts/sync/carrier/status.csv"), index_col="RECTIME")
        # 更改属性值
        npIndexes = pdDfSectionStatus["DEMOD_CARRIERLOCK"].values
        npResult = np.zeros(npIndexes.size)
        npResult[npIndexes == 2] = 1
        pdDfSectionStatus["DEMOD_CARRIERLOCK"] = npResult
        
        strRegularizedSamplesDir = os.path.join(strDemodDir, "parts/sync/carrier/regularized")
        if os.path.exists(strRegularizedSamplesDir):
            shutil.rmtree(strRegularizedSamplesDir)
        os.mkdir(strRegularizedSamplesDir)
        pdDfSectionStatus.to_csv(os.path.join(strRegularizedSamplesDir, "status.csv"), index_label="RECTIME")

### 生成样本

In [135]:
for strJob in os.listdir(strJobsDir):
    strJobDir = os.path.join(strJobsDir, strJob)
    listDemodDirs = fn_getDemodDirsOfAJob(strJobDir)
    for strDemodDir in listDemodDirs:
        strSectionDir = os.path.join(strDemodDir, "parts/sync/carrier/regularized")
        strLastSamplesDir = os.path.join(strDemodDir, "parts/sync/input/samples")
        
        # 清空样本文件夹
        strSamplesDir = os.path.join(strDemodDir, "parts/sync/carrier/samples")        
        if os.path.exists(strSamplesDir):
            shutil.rmtree(strSamplesDir)
        os.mkdir(strSamplesDir)
        
        # 正常训练样本
        os.makedirs(os.path.join(strSamplesDir, "normal/train"))
       
        
        # 异常样本
        os.mkdir(os.path.join(strSamplesDir, "abnormal"))
        pdDfAbnormalSamples = fn_concat(os.path.join(strLastSamplesDir, "abnormal/samples.csv"), \
               os.path.join(strSectionDir, "status.csv"))
        pdDfAbnormalSamples.to_csv(os.path.join(strSamplesDir, "abnormal/samples.csv"), index_label="RECTIME")

### 正则化

对载波同步部分的所有属性进行正则化，包括二元属性（二元属性正则化后不变）。存入preprocessed文件夹。读入所有载波同步正常训练样本，利用正则化器适应正常训练样本。转化所有样本。

In [181]:
listinputNormFeatures = ["DEMOD_IFLEVEL", "DEMOD_EBNOVALUE"]
listcarrierNormFeatures = ["DEMOD_CARRIEROFFSET"]
listNormFeatures = listinputNormFeatures + listcarrierNormFeatures

In [140]:
listNormalTrainingSamples = []
for strJob in os.listdir(strJobsDir):
    strJobDir = os.path.join(strJobsDir, strJob)
    listDemodDirs = fn_getDemodDirsOfAJob(strJobDir)
    for strDemodDir in listDemodDirs:
        pdDfNormalTrainingSamples = pd.read_csv(os.path.join(strDemodDir, \
                                            "parts/sync/carrier/samples/normal/train/samples.csv"),\
                                            index_col="RECTIME")
        listNormalTrainingSamples.append(pdDfNormalTrainingSamples[listNormFeatures])
pdDfNormalTrainingSamples = pd.concat(listNormalTrainingSamples)

oMinMaxScaler = sklearn.preprocessing.MinMaxScaler()
oMinMaxScaler.fit(pdDfNormalTrainingSamples)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [141]:
for strJob in os.listdir(strJobsDir):
    strJobDir = os.path.join(strJobsDir, strJob)
    listDemodDirs = fn_getDemodDirsOfAJob(strJobDir)
    for strDemodDir in listDemodDirs:
        # 载波同步部分的样本文件夹
        strSamplesDir = os.path.join(strDemodDir, "parts/sync/carrier/samples")
        
        # 正则化正常训练集
        # 清空预处理文件夹
        strPreprocessedNormalTrainingSamplesDir = os.path.join(strSamplesDir, "normal/train/preprocessed")
        if os.path.exists(strPreprocessedNormalTrainingSamplesDir):
            shutil.rmtree(strPreprocessedNormalTrainingSamplesDir)
        os.mkdir(strPreprocessedNormalTrainingSamplesDir)
        pdDfNormalTrainingSamples = pd.read_csv(os.path.join(strSamplesDir, "normal/train/samples.csv"),\
                                                    index_col="RECTIME")
        if not pdDfNormalTrainingSamples.empty:
            pdDfNormalTrainingSamples.loc[:, listNormFeatures] = \
                oMinMaxScaler.transform(pdDfNormalTrainingSamples.loc[:, listNormFeatures])
        """输出到磁盘的时候应该去除index，因为后面的tf在读取训练数据时不能识别字符串数据"""
        pdDfNormalTrainingSamples.to_csv(\
              os.path.join(strPreprocessedNormalTrainingSamplesDir, "samples.csv"), \
               index=False)
        
        # 正则化异常样本集
        # 清空预处理文件夹
        strPreprocessedAbnormalSamplesDir = os.path.join(strSamplesDir, "abnormal/preprocessed")
        if os.path.exists(strPreprocessedAbnormalSamplesDir):
            shutil.rmtree(strPreprocessedAbnormalSamplesDir)
        os.mkdir(strPreprocessedAbnormalSamplesDir)
        pdDfAbnormalSamples = pd.read_csv(os.path.join(strSamplesDir, "abnormal/samples.csv"), \
                                                    index_col="RECTIME")
        if not pdDfAbnormalSamples.empty:
            pdDfAbnormalSamples.loc[:, listNormFeatures] = \
                oMinMaxScaler.transform(pdDfAbnormalSamples.loc[:, listNormFeatures])
        """输出到磁盘的时候保留index"""
        pdDfAbnormalSamples.to_csv(\
            os.path.join(strPreprocessedAbnormalSamplesDir, "samples.csv"),\
            index_label="RECTIME")

## 比特同步
比特同步的特征参数：DEMOD_BITRATE（比特率）、DEMOD_BITRATEOFFSET（比特偏移）、DEMOD_BITRATEOFFSET（比特偏移Q通道）、DEMOD_BITLOCK（比特锁）、DEMOD_BITLOCKQCHL（比特锁Q通道）、DEMOD_TOTALBITNUMBER（总比特数）、DEMOD_TOTALBITNUMBERQCHL（总比特数Q通道）

由于已经有了和比特率相关的参数，所以就不考虑比特数的参数（"DEMOD_TOTALBITNUMBER", "DEMOD_TOTALBITNUMBERQCHL"）了，因为意义相同。

本来对于比特数这个参数，需要用当前秒的比特数减去上一秒的比特数，来得到当前秒接收的比特数。但是减完之后发现，有一些上报秒被的比特数被减成了零。感觉这个参数存在问题，故舍弃。

比特率是一个预设值。比特锁属性0为锁上，1为没锁。所以要把它们regularize一下。总比特数是一个累加值，要注意做差分。

### 载入数据
存储比特同步增加的属性，更新总的属性。

In [177]:
listbitFeatures = ["DEMOD_BITRATE", "DEMOD_BITRATEQCHL", "DEMOD_BITRATEOFFSET", "DEMOD_BITRATEOFFSETQCHL",\
                  "DEMOD_BITLOCK", "DEMOD_BITLOCKQCHL"]
listFeatures = listbitFeatures.copy()

In [143]:
for strJob in os.listdir(strJobsDir):
    strJobDir = os.path.join(strJobsDir, strJob)
    listDemodDirs = fn_getDemodDirsOfAJob(strJobDir)
    for strDemodDir in listDemodDirs:
        strBitDir = os.path.join(strDemodDir, "parts/sync/bit")
        if os.path.exists(strBitDir):
            shutil.rmtree(strBitDir)
        os.mkdir(strBitDir)
        
        strValidStatusFile = os.path.join(strDemodDir, "valid/status.csv")
        pdDfValidStatus = pd.read_csv(strValidStatusFile, index_col="RECTIME")
        pdDfBitStatus = pdDfValidStatus[listFeatures]
        pdDfBitStatus.to_csv(os.path.join(strBitDir, "status.csv"), index_label="RECTIME")

### 更改二元属性的值
按照载波同步的方法更改载波锁的二元属性值

比特锁属性0为锁上，1为没锁。要把比特锁属性值为0改为1，其它改为0

In [144]:
for strJob in os.listdir(strJobsDir):
    strJobDir = os.path.join(strJobsDir, strJob)
    listDemodDirs = fn_getDemodDirsOfAJob(strJobDir)
    for strDemodDir in listDemodDirs:
        strSectionDir = os.path.join(strDemodDir, "parts/sync/bit")        
        pdDfSectionStatus = pd.read_csv(os.path.join(strSectionDir, "status.csv"), index_col="RECTIME")
        
        # 更改bitlock和bitlcokqchl的值
        npIndexes = pdDfSectionStatus["DEMOD_BITLOCK"].values
        npResult = np.zeros(npIndexes.size)
        npResult[npIndexes == 0] = 1
        pdDfSectionStatus["DEMOD_BITLOCK"] = npResult
        npIndexes = pdDfSectionStatus["DEMOD_BITLOCKQCHL"].values
        npResult = np.zeros(npIndexes.size)
        npResult[npIndexes == 0] = 1
        pdDfSectionStatus["DEMOD_BITLOCKQCHL"] = npResult
        
        strRegularizedStatusDir = os.path.join(strSectionDir, "regularized")
        if os.path.exists(strRegularizedStatusDir):
            shutil.rmtree(strRegularizedStatusDir)
        os.mkdir(strRegularizedStatusDir)
        pdDfSectionStatus.to_csv(os.path.join(strRegularizedStatusDir, "status.csv"), index_label="RECTIME")

### 生成样本

In [145]:
for strJob in os.listdir(strJobsDir):
    strJobDir = os.path.join(strJobsDir, strJob)
    listDemodDirs = fn_getDemodDirsOfAJob(strJobDir)
    for strDemodDir in listDemodDirs:
        strSectionDir = os.path.join(strDemodDir, "parts/sync/bit/regularized")
        strLastSamplesDir = os.path.join(strDemodDir, "parts/sync/carrier/samples")
        
        # 清空样本文件夹
        strSamplesDir = os.path.join(strDemodDir, "parts/sync/bit/samples")        
        if os.path.exists(strSamplesDir):
            shutil.rmtree(strSamplesDir)
        os.mkdir(strSamplesDir)
        
        # 正常训练样本
        os.makedirs(os.path.join(strSamplesDir, "normal/train"))
        pdDfNormalSamples = fn_concat(os.path.join(strLastSamplesDir, "normal/train/samples.csv"), \
              os.path.join(strSectionDir, "status.csv"))
        pdDfNormalSamples.to_csv(os.path.join(strSamplesDir, "normal/train/samples.csv"), index_label="RECTIME")
        
        # 异常样本
        os.mkdir(os.path.join(strSamplesDir, "abnormal"))
        pdDfAbnormalSamples = fn_concat(os.path.join(strLastSamplesDir, "abnormal/samples.csv"), \
               os.path.join(strSectionDir, "status.csv"))
        pdDfAbnormalSamples.to_csv(os.path.join(strSamplesDir, "abnormal/samples.csv"), index_label="RECTIME")

### 正则化

对于二元变量要单独处理。因为如果训练集里所有的某二元变量的值均为1，那么，当用这个训练集训练过后的scaler去转化其它数据集的时候，其他数据集的1变量会被全部转化为0

In [182]:
listbitNormFeatures = ["DEMOD_BITRATE", "DEMOD_BITRATEQCHL", "DEMOD_BITRATEOFFSET", "DEMOD_BITRATEOFFSETQCHL"]
listNormFeatures = listinputNormFeatures + listcarrierNormFeatures + listbitNormFeatures

In [147]:
listNormalTrainingSamples = []
for strJob in os.listdir(strJobsDir):
    strJobDir = os.path.join(strJobsDir, strJob)
    listDemodDirs = fn_getDemodDirsOfAJob(strJobDir)
    for strDemodDir in listDemodDirs:
        pdDfNormalTrainingSamples = pd.read_csv(os.path.join(strDemodDir, \
                                            "parts/sync/bit/samples/normal/train/samples.csv"),\
                                            index_col="RECTIME")
        listNormalTrainingSamples.append(pdDfNormalTrainingSamples[listNormFeatures])
        
pdDfNormalTrainingSamples = pd.concat(listNormalTrainingSamples)
oMinMaxScaler = sklearn.preprocessing.MinMaxScaler()
oMinMaxScaler.fit(pdDfNormalTrainingSamples)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [148]:
for strJob in os.listdir(strJobsDir):
    strJobDir = os.path.join(strJobsDir, strJob)
    listDemodDirs = fn_getDemodDirsOfAJob(strJobDir)
    for strDemodDir in listDemodDirs:
        # 比特同步部分的样本文件夹
        strSamplesDir = os.path.join(strDemodDir, "parts/sync/bit/samples")
        
        # 正则化正常训练集
        # 清空预处理文件夹
        strPreprocessedNormalTrainingSamplesDir = os.path.join(strSamplesDir, "normal/train/preprocessed")
        if os.path.exists(strPreprocessedNormalTrainingSamplesDir):
            shutil.rmtree(strPreprocessedNormalTrainingSamplesDir)
        os.mkdir(strPreprocessedNormalTrainingSamplesDir)
        pdDfNormalTrainingSamples = pd.read_csv(os.path.join(strSamplesDir, "normal/train/samples.csv"),\
                                                    index_col="RECTIME")
        if not pdDfNormalTrainingSamples.empty:
            pdDfNormalTrainingSamples.loc[:, listNormFeatures] = \
                oMinMaxScaler.transform(pdDfNormalTrainingSamples.loc[:, listNormFeatures])
        """输出到磁盘的时候应该去除index，因为后面的tf在读取训练数据时不能识别字符串数据"""
        pdDfNormalTrainingSamples.to_csv(\
              os.path.join(strPreprocessedNormalTrainingSamplesDir, "samples.csv"), \
               index=False)
        
        # 正则化异常样本集
        # 清空预处理文件夹
        strPreprocessedAbnormalSamplesDir = os.path.join(strSamplesDir, "abnormal/preprocessed")
        if os.path.exists(strPreprocessedAbnormalSamplesDir):
            shutil.rmtree(strPreprocessedAbnormalSamplesDir)
        os.mkdir(strPreprocessedAbnormalSamplesDir)
        pdDfAbnormalSamples = pd.read_csv(os.path.join(strSamplesDir, "abnormal/samples.csv"), \
                                                    index_col="RECTIME")
        if not pdDfAbnormalSamples.empty:
            pdDfAbnormalSamples.loc[:, listNormFeatures] = \
                oMinMaxScaler.transform(pdDfAbnormalSamples.loc[:, listNormFeatures])
        """输出到磁盘的时候保留index"""
        pdDfAbnormalSamples.to_csv(\
            os.path.join(strPreprocessedAbnormalSamplesDir, "samples.csv"),\
            index_label="RECTIME")

## 维特比译码
维特比译码具有两套参数，要让模型自己去考虑合路和分路的情况，所以把维特比译码部分的I路和Q路的参数作为一个整体看待。

维特比译码部分包含参数：DEMOD_VITERBIINPUT（维特比分路合路开关）、DEMOD_VITERBI1DECODER、DEMOD_VITERBI1LOCK。

DEMOD_VITERBIINPUT表示维特比合路或者分路的开关，它具有{0，1，2}三个可能的取值，需要进行“独热编码”。DEMOD_VITERBI1DECODER和DEMOD_VITERBI2DECODER是I路和Q路维特比译码的开关，因为它们已经为二元属性0、1，所以也不用更改它的二元属性值。


把独热编码之后的状态参数文件存入diff文件夹，执行完成后具有如下文件结构：
* parts/
    * sync/
        * vi/
            * status.csv

### 载入数据
清空vi文件夹和其下的I、Q文件夹。读入维特比部分的参数

In [178]:
listviFeatures = ["DEMOD_VITERBIINPUT", \
                   "DEMOD_VITERBI1DECODER",\
                   "DEMOD_VITERBI2DECODER",\
                   "DEMOD_VITERBI1LOCK", "DEMOD_VITERBI2LOCK"]
listFeatures = listviFeatures

In [150]:
for strJob in os.listdir(strJobsDir):
    strJobDir = os.path.join(strJobsDir, strJob)
    listDemodDirs = fn_getDemodDirsOfAJob(strJobDir)
    for strDemodDir in listDemodDirs:
        strStatusDir = os.path.join(strDemodDir, "parts/sync/vi")
        if os.path.exists(strStatusDir):
            shutil.rmtree(strStatusDir)
        os.mkdir(strStatusDir)
        
        pdDfValidStatus = pd.read_csv(os.path.join(strDemodDir, "valid/status.csv"), index_col="RECTIME")
        pdDfBitStatus = pdDfValidStatus[listFeatures]
        pdDfBitStatus.to_csv(os.path.join(strStatusDir, "status.csv"), index_label="RECTIME")

### 独热编码
独热编码DEMOD_VITERBIINPUT，对应关系如下：
* 0---000 
* 1---010
* 2---001

每个编码位被重新命名，分别命名为novi、twovi、onevi

该步进行完成后，文件结构如下：
* onehot/
    * status.csv

In [241]:
for strJob in os.listdir(strJobsDir):
    for strDemodDir in fn_getDemodDirsOfAJob(os.path.join(strJobsDir, strJob)):
        strSectionDir = os.path.join(strDemodDir, "parts/sync/vi")
        strOneHotDir = os.path.join(strSectionDir, "onehot")
        if os.path.exists(strOneHotDir):
            shutil.rmtree(strOneHotDir)
        os.mkdir(strOneHotDir)
        
        pdDfStatus = pd.read_csv(os.path.join(strSectionDir, "status.csv"), index_col="RECTIME")
        
        pdDfStatus["novi"] = (pdDfStatus["DEMOD_VITERBIINPUT"] == 0).astype("int32")
        pdDfStatus["onevi"] = (pdDfStatus["DEMOD_VITERBIINPUT"] == 1).astype("int32")        
        pdDfStatus["twovi"] = (pdDfStatus["DEMOD_VITERBIINPUT"] == 2).astype("int32")        
        
        pdDfStatus.drop("DEMOD_VITERBIINPUT", inplace = True, axis=1)
        
        pdDfStatus.to_csv(os.path.join(strOneHotDir, "status.csv"), index_label="RECTIME")

### 生成样本
从onehot文件夹里取出样本

In [242]:
for strJob in os.listdir(strJobsDir):
    strJobDir = os.path.join(strJobsDir, strJob)
    listDemodDirs = fn_getDemodDirsOfAJob(strJobDir)
    for strDemodDir in listDemodDirs:
        strSectionDir = os.path.join(strDemodDir, "parts/sync/vi/onehot")
        strLastSamplesDir = os.path.join(strDemodDir, "parts/sync/bit/samples")
        
        # 清空样本文件夹
        strSamplesDir = os.path.join(strDemodDir, "parts/sync/vi/samples")        
        if os.path.exists(strSamplesDir):
            shutil.rmtree(strSamplesDir)
        os.mkdir(strSamplesDir)
        
        # 正常训练样本
        os.makedirs(os.path.join(strSamplesDir, "normal/train"))
        pdDfNormalSamples = fn_concat(os.path.join(strLastSamplesDir, "normal/train/samples.csv"), \
              os.path.join(strSectionDir, "status.csv"))
        pdDfNormalSamples.to_csv(os.path.join(strSamplesDir, "normal/train/samples.csv"), index_label="RECTIME")
        
        # 异常样本
        os.mkdir(os.path.join(strSamplesDir, "abnormal"))
        pdDfAbnormalSamples = fn_concat(os.path.join(strLastSamplesDir, "abnormal/samples.csv"), \
               os.path.join(strSectionDir, "status.csv"))
        pdDfAbnormalSamples.to_csv(os.path.join(strSamplesDir, "abnormal/samples.csv"), index_label="RECTIME")

### 正则化

In [183]:
listviNormFeatures = []
listNormFeatures = listinputNormFeatures + listcarrierNormFeatures + listbitNormFeatures + listviNormFeatures

In [244]:
listNormalTrainingSamples = []
for strJob in os.listdir(strJobsDir):
    strJobDir = os.path.join(strJobsDir, strJob)
    listDemodDirs = fn_getDemodDirsOfAJob(strJobDir)
    for strDemodDir in listDemodDirs:
        pdDfNormalTrainingSamples = pd.read_csv(os.path.join(strDemodDir, \
                                            "parts/sync/vi/samples/normal/train/samples.csv"),\
                                            index_col="RECTIME")
        listNormalTrainingSamples.append(pdDfNormalTrainingSamples[listNormFeatures])
        
pdDfNormalTrainingSamples = pd.concat(listNormalTrainingSamples)
oMinMaxScaler = sklearn.preprocessing.MinMaxScaler()
oMinMaxScaler.fit(pdDfNormalTrainingSamples)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [245]:
for strJob in os.listdir(strJobsDir):
    strJobDir = os.path.join(strJobsDir, strJob)
    listDemodDirs = fn_getDemodDirsOfAJob(strJobDir)
    for strDemodDir in listDemodDirs:
        # 维特比译码的样本文件夹
        strSamplesDir = os.path.join(strDemodDir, "parts/sync/vi/samples")
        
        # 正则化正常训练集
        # 清空预处理文件夹
        strPreprocessedNormalTrainingSamplesDir = os.path.join(strSamplesDir, "normal/train/preprocessed")
        if os.path.exists(strPreprocessedNormalTrainingSamplesDir):
            shutil.rmtree(strPreprocessedNormalTrainingSamplesDir)
        os.mkdir(strPreprocessedNormalTrainingSamplesDir)
        pdDfNormalTrainingSamples = pd.read_csv(os.path.join(strSamplesDir, "normal/train/samples.csv"),\
                                                    index_col="RECTIME")
        if not pdDfNormalTrainingSamples.empty:
            pdDfNormalTrainingSamples.loc[:, listNormFeatures] = \
                oMinMaxScaler.transform(pdDfNormalTrainingSamples.loc[:, listNormFeatures])
        """输出到磁盘的时候应该去除index，因为后面的tf在读取训练数据时不能识别字符串数据"""
        pdDfNormalTrainingSamples.to_csv(\
              os.path.join(strPreprocessedNormalTrainingSamplesDir, "samples.csv"), \
               index=False)
        
        # 正则化异常样本集
        # 清空预处理文件夹
        strPreprocessedAbnormalSamplesDir = os.path.join(strSamplesDir, "abnormal/preprocessed")
        if os.path.exists(strPreprocessedAbnormalSamplesDir):
            shutil.rmtree(strPreprocessedAbnormalSamplesDir)
        os.mkdir(strPreprocessedAbnormalSamplesDir)
        pdDfAbnormalSamples = pd.read_csv(os.path.join(strSamplesDir, "abnormal/samples.csv"), \
                                                    index_col="RECTIME")
        if not pdDfAbnormalSamples.empty:
            pdDfAbnormalSamples.loc[:, listNormFeatures] = \
                oMinMaxScaler.transform(pdDfAbnormalSamples.loc[:, listNormFeatures])
        """输出到磁盘的时候保留index"""
        pdDfAbnormalSamples.to_csv(\
            os.path.join(strPreprocessedAbnormalSamplesDir, "samples.csv"),\
            index_label="RECTIME")

## 帧同步

### 载入数据
在parts/sync下新建frame文件夹。将数据存入frame中。该步结束后，文件结构如下：
* parts/
    * sync/
        * frame/
            * status.csv

In [179]:
listframeFeatures = ["DEMOD_FRAMESYNCINPUT",\
                    "DPU_FRAMELEN1", "DPU_FRAMEHEADLEN1", "DPU_RECEIVEDFRAMECOUNTER1", \
                     "DPU_DROPOUTFRAMECOUNTER1", \
                     "DPU_FRAMELEN2", "DPU_FRAMEHEADLEN2", "DPU_RECEIVEDFRAMECOUNTER2", \
                     "DPU_DROPOUTFRAMECOUNTER2"]
listFeatures = listframeFeatures

In [236]:
for strJob in os.listdir(strJobsDir):
    strJobDir = os.path.join(strJobsDir, strJob)
    listDemodDirs = fn_getDemodDirsOfAJob(strJobDir)
    for strDemodDir in listDemodDirs:
        strStatusDir = os.path.join(strDemodDir, "parts/sync/frame")
        if os.path.exists(strStatusDir):
            shutil.rmtree(strStatusDir)
        os.mkdir(strStatusDir)
        
        pdDfValidStatus = pd.read_csv(os.path.join(strDemodDir, "valid/status.csv"), index_col="RECTIME")
        pdDfValidStatus[listFeatures].to_csv(os.path.join(strStatusDir, "status.csv"), index_label="RECTIME")

### 计算每秒的增加量
该步完成后，文件结构如下：
* frame/
    * status.csv
    * diff/
        * status.csv

In [237]:
listframeAggreFeatures = ["DPU_RECEIVEDFRAMECOUNTER1", "DPU_DROPOUTFRAMECOUNTER1", \
                          "DPU_RECEIVEDFRAMECOUNTER2", "DPU_DROPOUTFRAMECOUNTER2"]
listAggreFeatures = listframeAggreFeatures

In [238]:
def fn_subNumber(nWin):
        if nWin.size == 1:
            return  nWin[0]
        elif nWin[1] < nWin[0]:
                return nWin[1]
        else:
                return nWin[1] - nWin[0]

# 把0元素全部改为它的上一个元素的值
def fn_fillZeros(nWin):
    if nWin.size == 1:
        return nWin[0]
    elif nWin[1] == 0:
        return nWin[0]
    else:
        return nWin[1]

# 对比特数属性存入diff
for strJob in os.listdir(strJobsDir):
    strJobDir = os.path.join(strJobsDir, strJob)
    listDemodDirs = fn_getDemodDirsOfAJob(strJobDir)
    for strDemodDir in listDemodDirs:
        # 读入原始状态参数
        pdDfRawStatus = pd.read_csv(os.path.join(strDemodDir, "raw/status.csv"), index_col="RECTIME")
        strStatusDir = os.path.join(strDemodDir, "parts/sync/frame")
        # 清空diff
        strDiffDir = os.path.join(strStatusDir, "diff")
        if os.path.exists(strDiffDir):
            shutil.rmtree(strDiffDir)
        os.mkdir(strDiffDir)
        pdDfStatus = pd.read_csv(os.path.join(strStatusDir, "status.csv"), index_col="RECTIME")
        # 从第二个有效上报时间点开始，算每个点单独的包数
        pdDfStatus[listAggreFeatures] = pdDfStatus[listAggreFeatures].rolling(window=2, min_periods=1).\
        apply(fn_subNumber, raw=True)
        # 找到第一个有效时间点之前的时间点的整数坐标
        strFirstValidIndex = pdDfStatus.index[0]
        i = 0
        for strIndex in pdDfRawStatus.index:
            if strIndex == strFirstValidIndex:
                # 取出第一个有效时间点之前的记录和第一个有效时间点记录
                pdSeriesRaw = pdDfRawStatus.loc[pdDfRawStatus.index[i - 1], listAggreFeatures]
                pdSeriesToBeCulled = pdDfStatus.loc[strFirstValidIndex, listAggreFeatures]
                for j in range(len(pdSeriesToBeCulled)):
                    # 如果第一个有效点记录的在某域上的值大于等于相应的第一个有效时间点之前的记录的值，则做差
                    pdSeriesToBeCulled[j] = pdSeriesToBeCulled[j] - pdSeriesRaw[j] \
                        if pdSeriesToBeCulled[j] >= pdSeriesRaw[j] else pdSeriesToBeCulled[j]
                pdDfStatus.loc[strFirstValidIndex, listAggreFeatures] = pdSeriesToBeCulled
            i += 1
        # 填充0值
        pdDfStatus[listAggreFeatures] = pdDfStatus[listAggreFeatures].rolling(window=2, min_periods=1).\
        apply(fn_fillZeros, raw=True)
        pdDfStatus.to_csv(os.path.join(strDiffDir, "status.csv"), index_label="RECTIME")

### 独热编码
DEMOD_FRAMESYNCINPUT需要独热编码，对应关系如下：
* 0---100 noframe
* 1---010 oneframe
* 2---001 twoframe

In [248]:
for strJob in os.listdir(strJobsDir):
    for strDemodDir in fn_getDemodDirsOfAJob(os.path.join(strJobsDir, strJob)):
        strSectionDir = os.path.join(strDemodDir, "parts/sync/frame")
        strOneHotDir = os.path.join(strSectionDir, "onehot")
        if os.path.exists(strOneHotDir):
            shutil.rmtree(strOneHotDir)
        os.mkdir(strOneHotDir)
        
        pdDfStatus = pd.read_csv(os.path.join(strSectionDir, "diff/status.csv"), index_col="RECTIME")
        
        pdDfStatus["noframe"] = (pdDfStatus["DEMOD_FRAMESYNCINPUT"] == 0).astype("int32")
        pdDfStatus["oneframe"] = (pdDfStatus["DEMOD_FRAMESYNCINPUT"] == 1).astype("int32")        
        pdDfStatus["twoframe"] = (pdDfStatus["DEMOD_FRAMESYNCINPUT"] == 2).astype("int32")        
        
        pdDfStatus.drop("DEMOD_FRAMESYNCINPUT", inplace = True, axis=1)
        
        pdDfStatus.to_csv(os.path.join(strOneHotDir, "status.csv"), index_label="RECTIME")

### 生成样本

In [255]:
for strJob in os.listdir(strJobsDir):
    strJobDir = os.path.join(strJobsDir, strJob)
    listDemodDirs = fn_getDemodDirsOfAJob(strJobDir)
    for strDemodDir in listDemodDirs:
        strSectionDir = os.path.join(strDemodDir, "parts/sync/frame/onehot")
        strLastSamplesDir = os.path.join(strDemodDir, "parts/sync/vi/samples")
        
        # 清空样本文件夹
        strSamplesDir = os.path.join(strDemodDir, "parts/sync/frame/samples")        
        if os.path.exists(strSamplesDir):
            shutil.rmtree(strSamplesDir)
        os.mkdir(strSamplesDir)
        
        # 正常训练样本
        os.makedirs(os.path.join(strSamplesDir, "normal/train"))
        pdDfNormalSamples = fn_concat(os.path.join(strLastSamplesDir, "normal/train/samples.csv"), \
              os.path.join(strSectionDir, "status.csv"))
        pdDfNormalSamples.to_csv(os.path.join(strSamplesDir, "normal/train/samples.csv"), index_label="RECTIME")
        
        # 异常样本
        os.mkdir(os.path.join(strSamplesDir, "abnormal"))
        pdDfAbnormalSamples = fn_concat(os.path.join(strLastSamplesDir, "abnormal/samples.csv"), \
               os.path.join(strSectionDir, "status.csv"))
        pdDfAbnormalSamples.to_csv(os.path.join(strSamplesDir, "abnormal/samples.csv"), index_label="RECTIME")

### 正则化

In [184]:
listframeNormFeatures = ["DPU_FRAMELEN1", "DPU_FRAMEHEADLEN1", "DPU_RECEIVEDFRAMECOUNTER1", \
                         "DPU_DROPOUTFRAMECOUNTER1", \
                         "DPU_FRAMELEN2", "DPU_FRAMEHEADLEN2", "DPU_RECEIVEDFRAMECOUNTER2", \
                         "DPU_DROPOUTFRAMECOUNTER2"]
listNormFeatures = listinputNormFeatures + listcarrierNormFeatures + listbitNormFeatures + listviNormFeatures + \
    listframeNormFeatures

In [185]:
listNormalTrainingSamples = []
for strJob in os.listdir(strJobsDir):
    strJobDir = os.path.join(strJobsDir, strJob)
    listDemodDirs = fn_getDemodDirsOfAJob(strJobDir)
    for strDemodDir in listDemodDirs:
        pdDfNormalTrainingSamples = pd.read_csv(os.path.join(strDemodDir, \
                                            "parts/sync/frame/samples/normal/train/samples.csv"),\
                                            index_col="RECTIME")
        listNormalTrainingSamples.append(pdDfNormalTrainingSamples[listNormFeatures])
        
pdDfNormalTrainingSamples = pd.concat(listNormalTrainingSamples)
oMinMaxScaler = sklearn.preprocessing.MinMaxScaler()
oMinMaxScaler.fit(pdDfNormalTrainingSamples)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [187]:
for strJob in os.listdir(strJobsDir):
    strJobDir = os.path.join(strJobsDir, strJob)
    listDemodDirs = fn_getDemodDirsOfAJob(strJobDir)
    for strDemodDir in listDemodDirs:
        # 维特比译码的样本文件夹
        strSamplesDir = os.path.join(strDemodDir, "parts/sync/frame/samples")
        
        # 正则化正常训练集
        # 清空预处理文件夹
        strPreprocessedNormalTrainingSamplesDir = os.path.join(strSamplesDir, "normal/train/preprocessed")
        if os.path.exists(strPreprocessedNormalTrainingSamplesDir):
            shutil.rmtree(strPreprocessedNormalTrainingSamplesDir)
        os.mkdir(strPreprocessedNormalTrainingSamplesDir)
        pdDfNormalTrainingSamples = pd.read_csv(os.path.join(strSamplesDir, "normal/train/samples.csv"),\
                                                    index_col="RECTIME")
        if not pdDfNormalTrainingSamples.empty:
            pdDfNormalTrainingSamples.loc[:, listNormFeatures] = \
                oMinMaxScaler.transform(pdDfNormalTrainingSamples.loc[:, listNormFeatures])
        """输出到磁盘的时候应该去除index，因为后面的tf在读取训练数据时不能识别字符串数据"""
        pdDfNormalTrainingSamples.to_csv(\
              os.path.join(strPreprocessedNormalTrainingSamplesDir, "samples.csv"), \
               index=False)
        
        # 正则化异常样本集
        # 清空预处理文件夹
        strPreprocessedAbnormalSamplesDir = os.path.join(strSamplesDir, "abnormal/preprocessed")
        if os.path.exists(strPreprocessedAbnormalSamplesDir):
            shutil.rmtree(strPreprocessedAbnormalSamplesDir)
        os.mkdir(strPreprocessedAbnormalSamplesDir)
        pdDfAbnormalSamples = pd.read_csv(os.path.join(strSamplesDir, "abnormal/samples.csv"), \
                                                    index_col="RECTIME")
        if not pdDfAbnormalSamples.empty:
            pdDfAbnormalSamples.loc[:, listNormFeatures] = \
                oMinMaxScaler.transform(pdDfAbnormalSamples.loc[:, listNormFeatures])
        """输出到磁盘的时候保留index"""
        pdDfAbnormalSamples.to_csv(\
            os.path.join(strPreprocessedAbnormalSamplesDir, "samples.csv"),\
            index_label="RECTIME")

### 构造不做帧同步的数据
执行完成后文件结构如下：
* frame/
    * samples/
        * normal/
            * train/
                * preprocessed/
                    * samples.csv
                    * samples_noframe.csv

In [188]:
"""需要置零的参数名"""
listSettingFeatures = ["DPU_RECEIVEDFRAMECOUNTER1", "DPU_DROPOUTFRAMECOUNTER1", \
                      "DPU_RECEIVEDFRAMECOUNTER2", "DPU_DROPOUTFRAMECOUNTER2"]

In [189]:
for strJob in os.listdir(strJobsDir):
    for strDemodDir in fn_getDemodDirsOfAJob(os.path.join(strJobsDir, strJob)):
        strSamplesDir = os.path.join(strDemodDir, "parts/sync/frame/samples/normal/train/preprocessed")
        
        pdDfSamples = pd.read_csv(os.path.join(strSamplesDir, "samples.csv"))
        pdDfSamples.loc[:, listSettingFeatures] = 0.
        
        pdDfSamples.to_csv(os.path.join(strSamplesDir, "samples_noframe.csv"), index=False)

# 帧同步后——译码和解扰部分
DPU_DESCRAMBLING1、DPU_DESCRAMBLINGPOLYNOMIAL1、DPU_DESCRAMBLINGPRESETVALUE1、DPU_DECODEPOS1、DPU_LDPCSTATUS1、DPU_RSSTATUS1、DPU_RSGOODFRAMECOUNTER1（rs和ldpc共用）、DPU_RSBADFRAMECOUNTER1（rs和ldpc共用）、DPU_RSERRORBITNUMBER1、DPU_CRCSTATUS1、DPU_CRCGOODFRAMECOUNTER1、DPU_CRCBADFRAMECOUNTER1

## 载入数据

In [203]:
listdecodeFeatures = ["DPU_DESCRAMBLING1", "DPU_DESCRAMBLINGPOLYNOMIAL1", "DPU_DESCRAMBLINGPRESETVALUE1",\
                     "DPU_DESCRAMBLING2", "DPU_DESCRAMBLINGPOLYNOMIAL2", "DPU_DESCRAMBLINGPRESETVALUE2",\
                      
                     "DPU_DECODEPOS1", "DPU_LDPCSTATUS1", "DPU_RSSTATUS1", "DPU_RSGOODFRAMECOUNTER1",\
                      "DPU_RSBADFRAMECOUNTER1", "DPU_RSERRORBITNUMBER1",\
                     "DPU_DECODEPOS2", "DPU_LDPCSTATUS2", "DPU_RSSTATUS2", "DPU_RSGOODFRAMECOUNTER2",\
                      "DPU_RSBADFRAMECOUNTER2", "DPU_RSERRORBITNUMBER2",\
                      
                     "DPU_CRCSTATUS1", "DPU_CRCGOODFRAMECOUNTER1", "DPU_CRCBADFRAMECOUNTER1",\
                     "DPU_CRCSTATUS2", "DPU_CRCGOODFRAMECOUNTER2", "DPU_CRCBADFRAMECOUNTER2"]
listFeatures = listdecodeFeatures.copy()

In [205]:
for strJob in os.listdir(strJobsDir):
    strJobDir = os.path.join(strJobsDir, strJob)
    listDemodDirs = fn_getDemodDirsOfAJob(strJobDir)
    for strDemodDir in listDemodDirs:
        strStatusDir = os.path.join(strDemodDir, "parts/decode")
        if os.path.exists(strStatusDir):
            shutil.rmtree(strStatusDir)
        os.mkdir(strStatusDir)
        
        pdDfValidStatus = pd.read_csv(os.path.join(strDemodDir, "valid/status.csv"), index_col="RECTIME")
        pdDfValidStatus[listFeatures].to_csv(os.path.join(strStatusDir, "status.csv"), index_label="RECTIME")

## 计算每秒的增加量

In [206]:
listdecodeAggreFeatures = ["DPU_RSGOODFRAMECOUNTER1", "DPU_RSBADFRAMECOUNTER1", "DPU_RSERRORBITNUMBER1",\
                          "DPU_CRCGOODFRAMECOUNTER1", "DPU_CRCBADFRAMECOUNTER1",\
                          "DPU_RSGOODFRAMECOUNTER2", "DPU_RSBADFRAMECOUNTER2", "DPU_RSERRORBITNUMBER2",\
                          "DPU_CRCGOODFRAMECOUNTER2", "DPU_CRCBADFRAMECOUNTER2"]
listAggreFeatures = listdecodeAggreFeatures

In [207]:
# 对比特数属性存入diff
for strJob in os.listdir(strJobsDir):
    strJobDir = os.path.join(strJobsDir, strJob)
    listDemodDirs = fn_getDemodDirsOfAJob(strJobDir)
    for strDemodDir in listDemodDirs:
        # 读入原始状态参数
        pdDfRawStatus = pd.read_csv(os.path.join(strDemodDir, "raw/status.csv"), index_col="RECTIME")
        strStatusDir = os.path.join(strDemodDir, "parts/decode")
        # 清空diff
        strDiffDir = os.path.join(strStatusDir, "diff")
        if os.path.exists(strDiffDir):
            shutil.rmtree(strDiffDir)
        os.mkdir(strDiffDir)
        pdDfStatus = pd.read_csv(os.path.join(strStatusDir, "status.csv"), index_col="RECTIME")
        # 从第二个有效上报时间点开始，算每个点单独的包数
        pdDfStatus[listAggreFeatures] = pdDfStatus[listAggreFeatures].rolling(window=2, min_periods=1).\
        apply(fn_subNumber, raw=True)
        # 找到第一个有效时间点之前的时间点的整数坐标
        strFirstValidIndex = pdDfStatus.index[0]
        i = 0
        for strIndex in pdDfRawStatus.index:
            if strIndex == strFirstValidIndex:
                # 取出第一个有效时间点之前的记录和第一个有效时间点记录
                pdSeriesRaw = pdDfRawStatus.loc[pdDfRawStatus.index[i - 1], listAggreFeatures]
                pdSeriesToBeCulled = pdDfStatus.loc[strFirstValidIndex, listAggreFeatures]
                for j in range(len(pdSeriesToBeCulled)):
                    # 如果第一个有效点记录的在某域上的值大于等于相应的第一个有效时间点之前的记录的值，则做差
                    pdSeriesToBeCulled[j] = pdSeriesToBeCulled[j] - pdSeriesRaw[j] \
                        if pdSeriesToBeCulled[j] >= pdSeriesRaw[j] else pdSeriesToBeCulled[j]
                pdDfStatus.loc[strFirstValidIndex, listAggreFeatures] = pdSeriesToBeCulled
            i += 1
        # 填充0值
        pdDfStatus[listAggreFeatures] = pdDfStatus[listAggreFeatures].rolling(window=2, min_periods=1).\
        apply(fn_fillZeros, raw=True)
        pdDfStatus.to_csv(os.path.join(strDiffDir, "status.csv"), index_label="RECTIME")

## 独热编码
DPU_DESCRAMBLING需要独热编码，它具有的值和对应关系如下：
{0, 1, 2, 3}
* 0---1000 descr0
* 1---0100 descr1
* 2---0010 descr2
* 3---0001 descr3

In [210]:
for strJob in os.listdir(strJobsDir):
    for strDemodDir in fn_getDemodDirsOfAJob(os.path.join(strJobsDir, strJob)):
        strSectionDir = os.path.join(strDemodDir, "parts/decode")
        strOneHotDir = os.path.join(strSectionDir, "onehot")
        if os.path.exists(strOneHotDir):
            shutil.rmtree(strOneHotDir)
        os.mkdir(strOneHotDir)
        
        pdDfStatus = pd.read_csv(os.path.join(strSectionDir, "diff/status.csv"), index_col="RECTIME")
        
        pdDfStatus["descr10"] = (pdDfStatus["DPU_DESCRAMBLING1"] == 0).astype("int32")
        pdDfStatus["descr11"] = (pdDfStatus["DPU_DESCRAMBLING1"] == 1).astype("int32")        
        pdDfStatus["descr12"] = (pdDfStatus["DPU_DESCRAMBLING1"] == 2).astype("int32")  
        pdDfStatus["descr13"] = (pdDfStatus["DPU_DESCRAMBLING1"] == 3).astype("int32")  

        pdDfStatus["descr20"] = (pdDfStatus["DPU_DESCRAMBLING2"] == 0).astype("int32")
        pdDfStatus["descr21"] = (pdDfStatus["DPU_DESCRAMBLING2"] == 1).astype("int32")        
        pdDfStatus["descr22"] = (pdDfStatus["DPU_DESCRAMBLING2"] == 2).astype("int32")     
        pdDfStatus["descr23"] = (pdDfStatus["DPU_DESCRAMBLING2"] == 3).astype("int32")     

        pdDfStatus.drop(["DPU_DESCRAMBLING1", "DPU_DESCRAMBLING2"], inplace = True, axis=1)
        
        pdDfStatus.to_csv(os.path.join(strOneHotDir, "status.csv"), index_label="RECTIME")

## 生成样本

In [215]:
for strJob in os.listdir(strJobsDir):
    strJobDir = os.path.join(strJobsDir, strJob)
    listDemodDirs = fn_getDemodDirsOfAJob(strJobDir)
    for strDemodDir in listDemodDirs:
        strSectionDir = os.path.join(strDemodDir, "parts/decode/onehot")
        strLastSamplesDir = os.path.join(strDemodDir, "parts/sync/input/samples")
        
        # 清空样本文件夹
        strSamplesDir = os.path.join(strDemodDir, "parts/decode/samples")        
        if os.path.exists(strSamplesDir):
            shutil.rmtree(strSamplesDir)
        os.mkdir(strSamplesDir)
        
        # 正常训练样本
        os.makedirs(os.path.join(strSamplesDir, "normal/train"))
        pdDfNormalSamples = fn_concat(os.path.join(strLastSamplesDir, "normal/train/samples.csv"), \
              os.path.join(strSectionDir, "status.csv"))
        pdDfNormalSamples.to_csv(os.path.join(strSamplesDir, "normal/train/samples.csv"), index_label="RECTIME")
        
        # 异常样本
        os.mkdir(os.path.join(strSamplesDir, "abnormal"))
        pdDfAbnormalSamples = fn_concat(os.path.join(strLastSamplesDir, "abnormal/samples.csv"), \
               os.path.join(strSectionDir, "status.csv"))
        pdDfAbnormalSamples.to_csv(os.path.join(strSamplesDir, "abnormal/samples.csv"), index_label="RECTIME")

## 正则化

In [218]:
listdecodeNormFeatures = ["DPU_DESCRAMBLINGPOLYNOMIAL1", "DPU_DESCRAMBLINGPRESETVALUE1",\
                         "DPU_DESCRAMBLINGPOLYNOMIAL2", "DPU_DESCRAMBLINGPRESETVALUE2",\
                          
                         "DPU_RSGOODFRAMECOUNTER1", "DPU_RSBADFRAMECOUNTER1", "DPU_RSERRORBITNUMBER1",\
                         "DPU_RSGOODFRAMECOUNTER2", "DPU_RSBADFRAMECOUNTER2", "DPU_RSERRORBITNUMBER2",\
                         
                         "DPU_CRCGOODFRAMECOUNTER1", "DPU_CRCBADFRAMECOUNTER1",\
                         "DPU_CRCGOODFRAMECOUNTER2", "DPU_CRCBADFRAMECOUNTER2"]
listNormFeatures = listinputNormFeatures + listdecodeNormFeatures

In [219]:
listNormalTrainingSamples = []
for strJob in os.listdir(strJobsDir):
    strJobDir = os.path.join(strJobsDir, strJob)
    listDemodDirs = fn_getDemodDirsOfAJob(strJobDir)
    for strDemodDir in listDemodDirs:
        pdDfNormalTrainingSamples = pd.read_csv(os.path.join(strDemodDir, \
                                            "parts/decode/samples/normal/train/samples.csv"),\
                                            index_col="RECTIME")
        listNormalTrainingSamples.append(pdDfNormalTrainingSamples[listNormFeatures])
        
pdDfNormalTrainingSamples = pd.concat(listNormalTrainingSamples)
oMinMaxScaler = sklearn.preprocessing.MinMaxScaler()
oMinMaxScaler.fit(pdDfNormalTrainingSamples)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [220]:
for strJob in os.listdir(strJobsDir):
    strJobDir = os.path.join(strJobsDir, strJob)
    listDemodDirs = fn_getDemodDirsOfAJob(strJobDir)
    for strDemodDir in listDemodDirs:
        strSamplesDir = os.path.join(strDemodDir, "parts/decode/samples")
        
        # 正则化正常训练集
        # 清空预处理文件夹
        strPreprocessedNormalTrainingSamplesDir = os.path.join(strSamplesDir, "normal/train/preprocessed")
        if os.path.exists(strPreprocessedNormalTrainingSamplesDir):
            shutil.rmtree(strPreprocessedNormalTrainingSamplesDir)
        os.mkdir(strPreprocessedNormalTrainingSamplesDir)
        pdDfNormalTrainingSamples = pd.read_csv(os.path.join(strSamplesDir, "normal/train/samples.csv"),\
                                                    index_col="RECTIME")
        if not pdDfNormalTrainingSamples.empty:
            pdDfNormalTrainingSamples.loc[:, listNormFeatures] = \
                oMinMaxScaler.transform(pdDfNormalTrainingSamples.loc[:, listNormFeatures])
        """输出到磁盘的时候应该去除index，因为后面的tf在读取训练数据时不能识别字符串数据"""
        pdDfNormalTrainingSamples.to_csv(\
              os.path.join(strPreprocessedNormalTrainingSamplesDir, "samples.csv"), \
               index=False)
        
        # 正则化异常样本集
        # 清空预处理文件夹
        strPreprocessedAbnormalSamplesDir = os.path.join(strSamplesDir, "abnormal/preprocessed")
        if os.path.exists(strPreprocessedAbnormalSamplesDir):
            shutil.rmtree(strPreprocessedAbnormalSamplesDir)
        os.mkdir(strPreprocessedAbnormalSamplesDir)
        pdDfAbnormalSamples = pd.read_csv(os.path.join(strSamplesDir, "abnormal/samples.csv"), \
                                                    index_col="RECTIME")
        if not pdDfAbnormalSamples.empty:
            pdDfAbnormalSamples.loc[:, listNormFeatures] = \
                oMinMaxScaler.transform(pdDfAbnormalSamples.loc[:, listNormFeatures])
        """输出到磁盘的时候保留index"""
        pdDfAbnormalSamples.to_csv(\
            os.path.join(strPreprocessedAbnormalSamplesDir, "samples.csv"),\
            index_label="RECTIME")

# 学习一下pdnp

In [76]:
from sklearn import preprocessing as pp
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import roc_auc_score, roc_curve, auc, mean_squared_error

In [33]:
"""读入投票csv"""
pdDfRatings = pd.read_csv("../ml-20m/ratings.csv")

In [38]:
pdDfRatings.userId = pdDfRatings.userId.astype("int")
pdDfRatings.movieId = pdDfRatings.movieId.astype("int")
pdDfRatings.rating = pdDfRatings.rating.astype("float")
pdDfRatings.timestamp = pdDfRatings.timestamp.apply(\
                            lambda s: datetime.datetime.utcfromtimestamp(s).strftime("%Y-%m-%d %H:%M:%S"))

In [46]:
pdIndexMovie = pdDfRatings.groupby("movieId").count().sort_values("rating", ascending=False)[:1000].index

In [50]:
pdDfRatings2 = pdDfRatings.loc[pdDfRatings.movieId.isin(pdIndexMovie), :]

In [56]:
pdIndexUser = pdDfRatings2.groupby("userId").count().sort_values("rating").sample(n=1000, random_state=2020).index

In [58]:
pdDfRatings3 =pdDfRatings2.loc[pdDfRatings2.userId.isin(pdIndexUser), :]

In [63]:
pdDfMovieIdMap = pd.DataFrame(data=pdDfRatings3.movieId.unique(), columns=["oldMovieId"])
pdDfMovieIdMap["newMovieId"] = pdDfMovieIdMap.index + 1

pdDfUserIdMap = pd.DataFrame(data=pdDfRatings3.userId.unique(), columns=["oldUserId"])
pdDfUserIdMap["newUserId"] = pdDfUserIdMap.index + 1

In [71]:
pdDfRatings3 = pdDfRatings3.merge(pdDfMovieIdMap, left_on="movieId", right_on="oldMovieId")

In [72]:
pdDfRatings3.drop(labels=["oldMovieId"], axis=1, inplace=True)

In [74]:
pdDfRatings3 = pdDfRatings3.merge(pdDfUserIdMap, left_on="userId", right_on="oldUserId")
pdDfRatings3.drop(labels=["oldUserId"], axis=1, inplace=True)

In [81]:
pdDfTraining, pdDfTesting = train_test_split(pdDfRatings3, test_size=0.1, shuffle=True, random_state=2020)

In [82]:
pdDfValidation, pdDfTesting = train_test_split(pdDfTesting, test_size=0.5, shuffle=True, random_state=2020)

In [83]:
g_nUsers = pdDfRatings3.userId.unique().shape[0]
g_nMovies = pdDfRatings3.movieId.unique().shape[0]
g_nRatings = pdDfRatings3.shape[0]

In [109]:
nparrayRatingsTraining = np.zeros((g_nUsers, g_nMovies))
for onamedtuple in pdDfTraining.itertuples():
    nparrayRatingsTraining[onamedtuple[6] - 1, onamedtuple[5] - 1] = onamedtuple[3]

In [112]:
nparrayRatingsTesting = np.zeros((g_nUsers, g_nMovies))
for onamedtuple in pdDfValidation.itertuples():
    nparrayRatingsTesting[onamedtuple[6] - 1, onamedtuple[5] - 1] = onamedtuple[3]

In [113]:
nparrayRatingsValidation = np.zeros((g_nUsers, g_nMovies))
for onamedtuple in pdDfValidation.itertuples():
    nparrayRatingsValidation[onamedtuple[6] - 1, onamedtuple[5] - 1] = onamedtuple[3]

In [114]:
nparrayLabels = nparrayRatingsValidation[nparrayRatingsValidation.nonzero()].flatten()

In [141]:
nparrayPreds = np.zeros((g_nUsers, g_nMovies))
i = 0
for nparray in nparrayRatingsValidation:
    nparrayPreds[i] = np.mean(nparray[nparray > 0]) if not nparray.size else 0
    i += 1

In [142]:
nparrayPreds = nparrayPreds[nparrayRatingsValidation.nonzero()].flatten()

In [143]:
fMeanSquareErr = mean_squared_error(nparrayPreds, nparrayLabels)